# Reproduce Brian Long's results with Starfish

The `brian_long_smFISH.zip` file needed to follow along with this notebook can be downloaded [here]()

This notebook walks through a work flow that reproduces the smFISH result for one field of view using the starfish package. 
It assumes that you have unzipped `brian_long_smFISH.zip` in the same directory as this notebook. Thus, you should see:

raw/
brian_long_smFISH.ipynb

In [4]:
import os
from starfish.io import Stack
import numpy as np

In [2]:
experiment_json = os.path.expanduser('./brian_long_smFISH/experiment.json')

In [ ]:
s = Stack()
s.read(experiment_json)

In [5]:
# brian's data is uint, but starfish loads it as floats. Cast it to int here, which does not cause any loss of precision. 
s.image._data = s.image._data.astype(np.uint16)